In [5]:
!pip install imutils
!pip install opencv-python

Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable


# Version1 이미지 생성

In [1]:
import cv2
import os

# 이미지 저장 폴더 생성
if not os.path.exists('version1'):
    os.makedirs('version1')

# 100개의 이미지 생성
for i in range(100):
    # 이미지 생성
    image = cv2.imread('sample/image.png')
    
    # 사각형 좌표 설정
    start_point = (i % 10 * 50 + 50, i // 10 * 50 + 50)  # 사각형의 시작 좌표
    end_point = (i % 10 * 50 + 200, i // 10 * 50 + 200)  # 사각형의 끝 좌표
    
    # 사각형 그리기
    color = (23, 125, 255)  # 사각형의 색상 BGR
    thickness = 10  # 선의 두께
    image_with_rectangle = cv2.rectangle(image, start_point, end_point, color, thickness)
    
    # 이미지 저장
    cv2.imwrite(f'version1/image_{i+1}.png', image_with_rectangle)

# Version2 이미지 생성

In [2]:
# 이미지 저장 폴더 생성
if not os.path.exists('version2'):
    os.makedirs('version2')

# 100개의 이미지 생성
for i in range(100):
    # 이미지 생성
    image = cv2.imread(f'version1/image_{i+1}.png')
    
    # 원 중심 좌표 설정
    center_coordinates = (i % 10 * 50 + 150, i // 10 * 50 + 150)  # 원의 중심 좌표
    # 원 반지름 설정
    radius = 10  # 원의 반지름
    
    # 사각형 그리기
    color = (0, 255, 0)  # 사각형의 색상 BGR
    thickness = 10  # 선의 두께
    image_with_circle = cv2.circle(image, center_coordinates, radius, color, thickness)
    
    # 이미지 저장
    cv2.imwrite(f'version2/image_{i+1}.png', image_with_circle)

# 버전 별 이미지 목록 리스트 저장

In [3]:
# version1 폴더 이미지 파일 리스트 저장
version1_files = os.listdir('version1')

# version2 폴더 이미지 파일 리스트 저장
version2_files = os.listdir('version2')

# version1에만 있는 파일 찾기
unique_to_version1 = [file for file in version1_files if file not in version2_files]

# version2에만 있는 파일 찾기
unique_to_version2 = [file for file in version2_files if file not in version1_files]

# 두 폴더에 모두 있는 파일 찾기
common_files = [file for file in version1_files if file in version2_files]

# 이미지 비교

In [7]:
from skimage.metrics import structural_similarity as compare_ssim
import imutils
import cv2
import os

# 데이터 저장을 위한 리스트 생성
file_names = []
version1_paths = []
version2_paths = []
ssim_scores = []
pass_fail = []

for file in common_files:
    # 이미지 경로 설정
    version1_image_path = os.path.join('version1', file)
    version2_image_path = os.path.join('version2', file)
    
    # 이미지 경로가 존재하는지 확인
    if not os.path.exists(version1_image_path):
        raise FileNotFoundError(f"첫 번째 이미지 경로가 존재하지 않습니다: {version1_image_path}")

    if not os.path.exists(version2_image_path):
        raise FileNotFoundError(f"두 번째 이미지 경로가 존재하지 않습니다: {version2_image_path}")

    # 이미지 로드
    imageA = cv2.imread(version1_image_path)
    imageB = cv2.imread(version2_image_path)
    imageC = imageA.copy() # 결과물 저장용

    # 이미지 로드 확인
    if imageA is None:
        raise ValueError(f"이미지를 로드할 수 없습니다: {version1_image_path}")

    if imageB is None:
        raise ValueError(f"이미지를 로드할 수 없습니다: {version2_image_path}")
        
    tempDiff=cv2.subtract(imageA,imageB)

    # 이미지를 그레이스케일로 변환
    grayA = cv2.cvtColor(imageA, cv2.COLOR_BGR2GRAY)
    grayB = cv2.cvtColor(imageB, cv2.COLOR_BGR2GRAY)

    # SSIM 계산
    (score, diff) = compare_ssim(grayA, grayB, full=True)
    diff = (diff * 255).astype("uint8")

    print("SSIM: {}".format(score))
    
    thresh = cv2.threshold(diff, 0, 255,
        cv2.THRESH_BINARY_INV | cv2.THRESH_OTSU)[1]
    
    #차이점 체크
    tempDiff[thresh==255]=[0,0,255]
    imageC[thresh==255]=[0,0,255]
    
    cv2.waitKey(0)

    # 차이 이미지를 저장할 폴더 경로
    diff_folder_path = "diff"

    # 폴더가 존재하지 않는 경우 생성
    if not os.path.exists(diff_folder_path):
        os.makedirs(diff_folder_path)

#     # 차이 이미지 저장 경로 설정
#     diff_image_path = os.path.join(diff_folder_path, "difference_image.png")

    # 파일 이름에서 확장자 제거
    file_name, file_extension = os.path.splitext(file)
    
    # 이미지 저장
    cv2.imwrite(f'diff/diff_{file_name}.png', imageC)

    # 모든 열린 창 닫기
    cv2.destroyAllWindows()
    
    # 데이터 리스트에 추가
    file_names.append(file)
    version1_paths.append(version1_image_path)
    version2_paths.append(version2_image_path)
    ssim_scores.append(score)

    # SSIM 수치에 따른 PASS/FAIL 결정
    if score >= 1.0:
        pass_fail.append("PASS")
    else:
        pass_fail.append("DIFF")

[[[0 0 0]
  [0 0 0]
  [0 0 0]
  ...
  [0 0 0]
  [0 0 0]
  [0 0 0]]

 [[0 0 0]
  [0 0 0]
  [0 0 0]
  ...
  [0 0 0]
  [0 0 0]
  [0 0 0]]

 [[0 0 0]
  [0 0 0]
  [0 0 0]
  ...
  [0 0 0]
  [0 0 0]
  [0 0 0]]

 ...

 [[0 0 0]
  [0 0 0]
  [0 0 0]
  ...
  [0 0 0]
  [0 0 0]
  [0 0 0]]

 [[0 0 0]
  [0 0 0]
  [0 0 0]
  ...
  [0 0 0]
  [0 0 0]
  [0 0 0]]

 [[0 0 0]
  [0 0 0]
  [0 0 0]
  ...
  [0 0 0]
  [0 0 0]
  [0 0 0]]]
SSIM: 0.9988410213207028
[[[0 0 0]
  [0 0 0]
  [0 0 0]
  ...
  [0 0 0]
  [0 0 0]
  [0 0 0]]

 [[0 0 0]
  [0 0 0]
  [0 0 0]
  ...
  [0 0 0]
  [0 0 0]
  [0 0 0]]

 [[0 0 0]
  [0 0 0]
  [0 0 0]
  ...
  [0 0 0]
  [0 0 0]
  [0 0 0]]

 ...

 [[0 0 0]
  [0 0 0]
  [0 0 0]
  ...
  [0 0 0]
  [0 0 0]
  [0 0 0]]

 [[0 0 0]
  [0 0 0]
  [0 0 0]
  ...
  [0 0 0]
  [0 0 0]
  [0 0 0]]

 [[0 0 0]
  [0 0 0]
  [0 0 0]
  ...
  [0 0 0]
  [0 0 0]
  [0 0 0]]]
SSIM: 0.9988664824154948
[[[0 0 0]
  [0 0 0]
  [0 0 0]
  ...
  [0 0 0]
  [0 0 0]
  [0 0 0]]

 [[0 0 0]
  [0 0 0]
  [0 0 0]
  ...
  [0 0 0]
  [0 0 0

SSIM: 0.9990725348719349
[[[0 0 0]
  [0 0 0]
  [0 0 0]
  ...
  [0 0 0]
  [0 0 0]
  [0 0 0]]

 [[0 0 0]
  [0 0 0]
  [0 0 0]
  ...
  [0 0 0]
  [0 0 0]
  [0 0 0]]

 [[0 0 0]
  [0 0 0]
  [0 0 0]
  ...
  [0 0 0]
  [0 0 0]
  [0 0 0]]

 ...

 [[0 0 0]
  [0 0 0]
  [0 0 0]
  ...
  [0 0 0]
  [0 0 0]
  [0 0 0]]

 [[0 0 0]
  [0 0 0]
  [0 0 0]
  ...
  [0 0 0]
  [0 0 0]
  [0 0 0]]

 [[0 0 0]
  [0 0 0]
  [0 0 0]
  ...
  [0 0 0]
  [0 0 0]
  [0 0 0]]]
SSIM: 0.9990882010135617
[[[0 0 0]
  [0 0 0]
  [0 0 0]
  ...
  [0 0 0]
  [0 0 0]
  [0 0 0]]

 [[0 0 0]
  [0 0 0]
  [0 0 0]
  ...
  [0 0 0]
  [0 0 0]
  [0 0 0]]

 [[0 0 0]
  [0 0 0]
  [0 0 0]
  ...
  [0 0 0]
  [0 0 0]
  [0 0 0]]

 ...

 [[0 0 0]
  [0 0 0]
  [0 0 0]
  ...
  [0 0 0]
  [0 0 0]
  [0 0 0]]

 [[0 0 0]
  [0 0 0]
  [0 0 0]
  ...
  [0 0 0]
  [0 0 0]
  [0 0 0]]

 [[0 0 0]
  [0 0 0]
  [0 0 0]
  ...
  [0 0 0]
  [0 0 0]
  [0 0 0]]]
SSIM: 0.9991138261494603
[[[0 0 0]
  [0 0 0]
  [0 0 0]
  ...
  [0 0 0]
  [0 0 0]
  [0 0 0]]

 [[0 0 0]
  [0 0 0]
  [0 0 0]

[[[0 0 0]
  [0 0 0]
  [0 0 0]
  ...
  [0 0 0]
  [0 0 0]
  [0 0 0]]

 [[0 0 0]
  [0 0 0]
  [0 0 0]
  ...
  [0 0 0]
  [0 0 0]
  [0 0 0]]

 [[0 0 0]
  [0 0 0]
  [0 0 0]
  ...
  [0 0 0]
  [0 0 0]
  [0 0 0]]

 ...

 [[0 0 0]
  [0 0 0]
  [0 0 0]
  ...
  [0 0 0]
  [0 0 0]
  [0 0 0]]

 [[0 0 0]
  [0 0 0]
  [0 0 0]
  ...
  [0 0 0]
  [0 0 0]
  [0 0 0]]

 [[0 0 0]
  [0 0 0]
  [0 0 0]
  ...
  [0 0 0]
  [0 0 0]
  [0 0 0]]]
SSIM: 0.9993543820326484
[[[0 0 0]
  [0 0 0]
  [0 0 0]
  ...
  [0 0 0]
  [0 0 0]
  [0 0 0]]

 [[0 0 0]
  [0 0 0]
  [0 0 0]
  ...
  [0 0 0]
  [0 0 0]
  [0 0 0]]

 [[0 0 0]
  [0 0 0]
  [0 0 0]
  ...
  [0 0 0]
  [0 0 0]
  [0 0 0]]

 ...

 [[0 0 0]
  [0 0 0]
  [0 0 0]
  ...
  [0 0 0]
  [0 0 0]
  [0 0 0]]

 [[0 0 0]
  [0 0 0]
  [0 0 0]
  ...
  [0 0 0]
  [0 0 0]
  [0 0 0]]

 [[0 0 0]
  [0 0 0]
  [0 0 0]
  ...
  [0 0 0]
  [0 0 0]
  [0 0 0]]]
SSIM: 0.9991784192565794
[[[0 0 0]
  [0 0 0]
  [0 0 0]
  ...
  [0 0 0]
  [0 0 0]
  [0 0 0]]

 [[0 0 0]
  [0 0 0]
  [0 0 0]
  ...
  [0 0 0]
  [0 0 0

SSIM: 0.9988927795079513
[[[0 0 0]
  [0 0 0]
  [0 0 0]
  ...
  [0 0 0]
  [0 0 0]
  [0 0 0]]

 [[0 0 0]
  [0 0 0]
  [0 0 0]
  ...
  [0 0 0]
  [0 0 0]
  [0 0 0]]

 [[0 0 0]
  [0 0 0]
  [0 0 0]
  ...
  [0 0 0]
  [0 0 0]
  [0 0 0]]

 ...

 [[0 0 0]
  [0 0 0]
  [0 0 0]
  ...
  [0 0 0]
  [0 0 0]
  [0 0 0]]

 [[0 0 0]
  [0 0 0]
  [0 0 0]
  ...
  [0 0 0]
  [0 0 0]
  [0 0 0]]

 [[0 0 0]
  [0 0 0]
  [0 0 0]
  ...
  [0 0 0]
  [0 0 0]
  [0 0 0]]]
SSIM: 0.9992396895367893
[[[0 0 0]
  [0 0 0]
  [0 0 0]
  ...
  [0 0 0]
  [0 0 0]
  [0 0 0]]

 [[0 0 0]
  [0 0 0]
  [0 0 0]
  ...
  [0 0 0]
  [0 0 0]
  [0 0 0]]

 [[0 0 0]
  [0 0 0]
  [0 0 0]
  ...
  [0 0 0]
  [0 0 0]
  [0 0 0]]

 ...

 [[0 0 0]
  [0 0 0]
  [0 0 0]
  ...
  [0 0 0]
  [0 0 0]
  [0 0 0]]

 [[0 0 0]
  [0 0 0]
  [0 0 0]
  ...
  [0 0 0]
  [0 0 0]
  [0 0 0]]

 [[0 0 0]
  [0 0 0]
  [0 0 0]
  ...
  [0 0 0]
  [0 0 0]
  [0 0 0]]]
SSIM: 0.9991639218904776
[[[0 0 0]
  [0 0 0]
  [0 0 0]
  ...
  [0 0 0]
  [0 0 0]
  [0 0 0]]

 [[0 0 0]
  [0 0 0]
  [0 0 0]

SSIM: 0.9987276144173868
[[[0 0 0]
  [0 0 0]
  [0 0 0]
  ...
  [0 0 0]
  [0 0 0]
  [0 0 0]]

 [[0 0 0]
  [0 0 0]
  [0 0 0]
  ...
  [0 0 0]
  [0 0 0]
  [0 0 0]]

 [[0 0 0]
  [0 0 0]
  [0 0 0]
  ...
  [0 0 0]
  [0 0 0]
  [0 0 0]]

 ...

 [[0 0 0]
  [0 0 0]
  [0 0 0]
  ...
  [0 0 0]
  [0 0 0]
  [0 0 0]]

 [[0 0 0]
  [0 0 0]
  [0 0 0]
  ...
  [0 0 0]
  [0 0 0]
  [0 0 0]]

 [[0 0 0]
  [0 0 0]
  [0 0 0]
  ...
  [0 0 0]
  [0 0 0]
  [0 0 0]]]
SSIM: 0.9987540439506114
[[[0 0 0]
  [0 0 0]
  [0 0 0]
  ...
  [0 0 0]
  [0 0 0]
  [0 0 0]]

 [[0 0 0]
  [0 0 0]
  [0 0 0]
  ...
  [0 0 0]
  [0 0 0]
  [0 0 0]]

 [[0 0 0]
  [0 0 0]
  [0 0 0]
  ...
  [0 0 0]
  [0 0 0]
  [0 0 0]]

 ...

 [[0 0 0]
  [0 0 0]
  [0 0 0]
  ...
  [0 0 0]
  [0 0 0]
  [0 0 0]]

 [[0 0 0]
  [0 0 0]
  [0 0 0]
  ...
  [0 0 0]
  [0 0 0]
  [0 0 0]]

 [[0 0 0]
  [0 0 0]
  [0 0 0]
  ...
  [0 0 0]
  [0 0 0]
  [0 0 0]]]
SSIM: 0.9989021699749053
[[[0 0 0]
  [0 0 0]
  [0 0 0]
  ...
  [0 0 0]
  [0 0 0]
  [0 0 0]]

 [[0 0 0]
  [0 0 0]
  [0 0 0]

SSIM: 0.9987128139572286
[[[0 0 0]
  [0 0 0]
  [0 0 0]
  ...
  [0 0 0]
  [0 0 0]
  [0 0 0]]

 [[0 0 0]
  [0 0 0]
  [0 0 0]
  ...
  [0 0 0]
  [0 0 0]
  [0 0 0]]

 [[0 0 0]
  [0 0 0]
  [0 0 0]
  ...
  [0 0 0]
  [0 0 0]
  [0 0 0]]

 ...

 [[0 0 0]
  [0 0 0]
  [0 0 0]
  ...
  [0 0 0]
  [0 0 0]
  [0 0 0]]

 [[0 0 0]
  [0 0 0]
  [0 0 0]
  ...
  [0 0 0]
  [0 0 0]
  [0 0 0]]

 [[0 0 0]
  [0 0 0]
  [0 0 0]
  ...
  [0 0 0]
  [0 0 0]
  [0 0 0]]]
SSIM: 0.998663974560124
[[[0 0 0]
  [0 0 0]
  [0 0 0]
  ...
  [0 0 0]
  [0 0 0]
  [0 0 0]]

 [[0 0 0]
  [0 0 0]
  [0 0 0]
  ...
  [0 0 0]
  [0 0 0]
  [0 0 0]]

 [[0 0 0]
  [0 0 0]
  [0 0 0]
  ...
  [0 0 0]
  [0 0 0]
  [0 0 0]]

 ...

 [[0 0 0]
  [0 0 0]
  [0 0 0]
  ...
  [0 0 0]
  [0 0 0]
  [0 0 0]]

 [[0 0 0]
  [0 0 0]
  [0 0 0]
  ...
  [0 0 0]
  [0 0 0]
  [0 0 0]]

 [[0 0 0]
  [0 0 0]
  [0 0 0]
  ...
  [0 0 0]
  [0 0 0]
  [0 0 0]]]
SSIM: 0.9987862735978067
[[[0 0 0]
  [0 0 0]
  [0 0 0]
  ...
  [0 0 0]
  [0 0 0]
  [0 0 0]]

 [[0 0 0]
  [0 0 0]
  [0 0 0]


# 데이터 저장

In [8]:
import pandas as pd

# 데이터프레임 생성
data = {
    "Version1 File": version1_paths,
    "Version2 File": version2_paths,
    "SSIM Score": ssim_scores,
    "PASS/FAIL": pass_fail
}
df = pd.DataFrame(data)

# 엑셀 파일로 저장
df.to_excel("image_comparison_results.xlsx", index=False)
print("엑셀 파일이 생성되었습니다.")

엑셀 파일이 생성되었습니다.
